<h1 style="color:blue"> Lisamaterjal: Twitteri veebiteenuste kasutamine Pythonis </h1>

Kui soovid Twitter'ist andmeid korjata ja analüüsida, ei ole tingimata vaja ehitada veebiämblikku: Twitter pakub välja veebiteenused, mille kaudu saab (programmaatiliselt) otsida ja koguda päevakajalisi säutse, uurida kasutajate säutsuvooge ja omavahelist suhtlust ning teha palju muudki.

Käesolevas lisamaterjalis uurimegi samm-sammult, kuidas saab Pythoni abil Twitteri säutse otsida ja alla laadida.

## Esimesed sammud

### I. Twitter'i konto loomine ja seadistamine

Selleks, et Twitter-is päringuid teha, peab olema Twitteri kasutajakonto. Konto saab luua aadressil: https://twitter.com/signup

Kui konto on olemas, siis tuleb sellega siduda mobiilinumber. Seejärel saadetakse lühisõnumiga kinnituskood, mille pead Twitter'i veebilehel sisestama. 
Alles siis, kui kontoga on mobiilinumber seotud, saad ligipääsu Twitteri veebiteenustele, st saad hakata Pythonis veebiteenustele päringuid tegema.

### II. Ligipääs veebiteenustele: _app_'i loomine ja võtmed

Kui oled oma kasutajaga Twitter-isse sisse loginud, mine lehele https://developer.twitter.com/en/portal/dashboard ja loo uus projekt / rakendus.

Kui rakenduse loomine on õnnestunud, peaksid kätte saama ligipääsuvõtmed, mille kaudu hakkad veebiteenustega suhtlema. 
Selleks peaksid pärast rakenduse loomist valima _"Key and Access Tokens"_ kopeerima sealt _"Consumer Key"_ ja _"Consumer Secret"_ väärtused. 
Allpool peaks veel olema paneel _"Your Access Token"_ ning kui oled värske veebiteenuse kasutaja, peaksid selle uuesti looma (nupp nimega _"create access token"_ vms). 
Kui isiklik ligipääsuvõti loodud, kopeeri ka sealt väärtused _"Access Token"_ ja _"Access Token Secret"_;

### III. _Python Twitter Tools_'i paigaldamine

Twitteri veebiteenusega suhtlemiseks on loodud üksjagu palju Pythoni teeke, nimekirja neist leiate nt [siit](https://developer.twitter.com/en/docs/twitter-api/tools-and-libraries). Käesolevas praktikumis proovime esmalt teeki [_Python Twitter Tools_](https://github.com/sixohsix/twitter). Teegi installimiseks tipi conda käsureale:

    pip install twitter    

### IV. Näidispäring

Esimese sammuna tuleb täita ligipääsuvõtmete muutujad:

In [ ]:
# Kasutajakonto ja rakendusega seotud ligipääsuvõtmed
# (TODO: täita oma võtmete väärtustega)
ACCESS_TOKEN    = ''
ACCESS_SECRET   = ''
CONSUMER_KEY    = ''
CONSUMER_SECRET = ''

Seejärel loome ühenduse:

In [ ]:
from twitter import Twitter, OAuth

# Loome veebiteenusega ühenduse ("logime sisse")
t = Twitter(
    auth=OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET))

Nüüd ongi võimalik teha päringuid. 

Otsime kõige värskemaid eestikeelseid säutse, mis sisaldavad sõna `Tartu`:

In [ ]:
results = t.search.tweets(q="Tartu", lang="et", result_type='recent', count=15)
print( "Leiti", len(results['statuses']),"säutsu." )

Suhtlus veebiteenusega toimub JSON formaadis andmeid vahetades, seega on tagastatav tulemus samuti mitmetasemeline sõnastik-andmestruktuur. Võid selle sisu uurida `pprint` abil. Leitud säutsud on saadaval võtme `'statuses'` all järjendina:

In [ ]:
from pprint import pprint

# Trükime välja esimese leitud säutsu andmed
pprint( results['statuses'][0] )

Enamik eeltoodud andmetest on tehnilist laadi ning ei pruugi olla väga huvitavad. Seega tasub teha üks tsükkel üle leitud säutsude ning noppida välja huvipakkuvaimad andmed. Näiteks nii:

In [ ]:
# Nopime välja huvipakkuvad andmed
for twnr, tweet_obj in enumerate(results['statuses']):
    # säutsu andmed
    time     = tweet_obj.get('created_at', '---')
    language = tweet_obj.get('lang', '---')
    text     = tweet_obj.get('text', '---')
    # säutsuja andmed
    user_name       = tweet_obj['user'].get('name', '---')
    user_screenname = tweet_obj['user'].get('screen_name', '---')
    user_location   = tweet_obj['user'].get('location', '---')
    user_timezone   = tweet_obj['user'].get('time_zone', '---')
    # väljastame
    print(str(twnr+1)+'. "'+text+'"')
    print('   user_screenname:',user_screenname)
    print('   user_name:      ',user_name)
    print('   time:           ',time)
    print('   time_zone:      ',user_timezone)
    print('   language:       ',language)
    print('   user location:  ',user_location)
    print()

## Keerulisemad päringud

### Suurem hulk vastuseid ja päringutulemustes navigeerimine

Eelmises näites piirati tagastatavate säutsude arv 15-le. Tagastatavate säutsude arvu (parameeter `count`) võib tõsta maksimaalselt kuni 100-ni ning kui leitakse sellest rohkem säutse, tuleb kõigi tulemuste kättesaamiseks hakata päringutulemustes navigeerima. 
Leitud säutsud on ajaliselt sorteeritud (kõige värskemad kõige ees) ning parameetriga `max_id` saab määrata, millisest (mineviku)säutsust alates tuleb valida 15 või 100 (ehk siis `count`) eelmist säutsu. 
Seega suure hulga tulemuste korral saab võtta viimase säutsu `id` ja luua selle põhjal uue `max_id` (`max_id = id - 1`) ning kasutada seda otsingul.

Järgmisel otsingul kasutatava `max_id` väärtus on tegelikult olemas ka päringu metaandmetes. Seega, kui päringule oli rohkem vasteid kui tagastati, siis ilmub metaandmetesse võti `next_results`, mille väärtuseks on järgmise päringu url koos `max_id`-iga. 

In [ ]:
# Otsime mõne globaalselt tuntud nime järgi, et saada palju vasteid
results = t.search.tweets(q="Putin", lang="et", result_type='recent', count=100) 
print( "Leiti", len(results['statuses']),"säutsu." )

In [ ]:
from pprint import pprint

# Uurime päringu metaandmeid: kas vasteid on veel?
pprint(results['search_metadata'])

Vasteid oli veel.

Seega võime teha programmi, mis sooritab järgemööda päringuid ja kogub säutsud kokku üle kõigi andmete:

In [ ]:
# Näide: päringutulemuste läbiketramine
import re

query = "Putin"
all_results = []
max_count = 100
results = t.search.tweets(q=query, lang="et", result_type='recent', count=max_count) 
while True:
    # Algväärtustame max_id (esialgu me ei tea, kas tulemusi on veel või mitte)
    max_id = None
    print( "Leiti", len(results['statuses']),"säutsu." )
    if results['statuses']:
        # Salvestame leitud säutsud
        all_results.extend( results['statuses'] )
        # Leiame max_id
        next_results_url = results['search_metadata'].get('next_results')
        if next_results_url:
            max_id_str = re.findall('max_id=(\d+)&', next_results_url)
            max_id = int(max_id_str[0])
    # Kui max_id on leitud, teeme uue päringu
    if max_id:
        results = t.search.tweets(q=query, lang="et", max_id=max_id, result_type='recent', count=max_count)
    else:
        # Kui max_id on leidmata, lõpetame
        break
print( "Kokku leiti päringule '"+query+"' vasteks",len(all_results),"säutsu." )

### Keeletuvastus

Kui leitud päringutulemusi detailsemalt uurida, siis tuleb esile ka üks keelespetsiifiliste päringute tegemise kitsaskoht. Nimelt on paljud säutsudest küll märgitud eestikeelseteks, ent tegelikult on nende sisu eesti keelest üsna kaugel (tegemist on kas mõne sugulaskeele või eesti keelega sarnaseid häälikutejadasid sisaldava säutsuga).

In [ ]:
# Uurime viimast 10-t säutsu
for i in range(10):
    tweet_obj = all_results[-i]
    # väljastame
    print(str(i+1)+'. "'+tweet_obj.get('text', '---')+'"')   

Üks võimalus probleemist ülesaamiseks (st eestikeelsete säutsude väljasõelumiseks) on rakendada tekstil morfoloogilist analüsaatorit ja uurida, kui suur osakaal sõnadest on tundmatud. Säutsud, kus enamik sõnu jääb morf analüsaatorile tundmatuks, võib suure tõenäosusega lugeda mitte-eestikeelseteks. 

Teine võimalus on lasta säutsud automaatsest keeletuvastusest läbi. Pythoni teek [LangID](https://github.com/saffsd/langid.py) sobib täitsa hästi sellesse rolli. Teegi installimiseks käsureale:

    pip install langid  
    
Edasi rakendame keeletuvastust eestikeelsete säutsude väljanoppimiseks:

In [ ]:
# Impordime keeletuvastuse teegi
import langid

all_results_et = []
for tweet_obj in all_results:
    text = tweet_obj.get('text', '---')
    # Tuvastame säutsu keele
    lang = langid.classify(text)[0]
    if lang == 'et':
        # Nopime välja vaid eestikeelsed säutsud
        all_results_et.append( tweet_obj )

print( "Automaatne keeletuvastus leidis ",len(all_results_et)," eestikeelset säutsu." )

Heidame tulemustele pilgu peale:

In [ ]:
# Uurime viimast 10-t säutsu
for i in range(10):
    tweet_obj = all_results_et[-i]
    # väljastame
    print(str(i+1)+'. "'+tweet_obj.get('text', '---')+'"')

Nagu näha, ei ole ka seekord tulemused päris perfektsed, aga prahti paistab siiski oluliselt vähem. Tulemusi saaks ehk veelgi täpsemaks kui uurida ka [keeletuvastuse tõenäosuseid](https://github.com/saffsd/langid.py#probability-normalization) ning võtta välja vaid kõige suurema tõenäosusmäärangu saanud säutsud.

### _Search_ liidese parameetrid ja piirangud otsimisel

Eelpool kasutasime otsinguliidest _Search_ (`t.search.tweets`). _Search_ võimaldab leida kõige värskemaid säutse (parameeter `result_type='recent'`), kõige populaarsemaid (`result_type='popular'`) ning segamini nii värskeid kui populaarseid (`result_type='mixed'`, vaikimisi). 

Kui otsid eestikeelseid säutse (`lang='et'`) ja kasutad eestikeelseid võtmesõnu, ei pruugi aga populaarsete päringule vastuseid leida, kuna eestikeelsed säutsud ei ulatu eriti globaalsel populaarsusskaalal konkureerima. 
Samuti on piiratud värskeimate säutsude otsimine: veebiliides tagastab vaid [viimase 7 päeva säutse](https://developer.twitter.com/en/docs/tweets/search/overview/standard).

Lisaks saab parameetritega täpsustada nt säutsuja asukoha (koordinaadid) ja kuupäeva (kitsendatud 1 nädala piires). Võimalike päringuparameetrite kohta vaata täpsemalt [lehelt](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets#parameters).

### (!) Oluline: Päringulimiidid

Veebiteenustele päringuid tehes tuleb jälgida, et ei ületaks Twitteri poolt seatud päringulimiite. Päringulimiitide ületamisel (või veel hullem -- serveri üleuputamisel päringutega) võidakse tühistada ligipääs veebiteenustele ja kasutajakonto. 

Vaikimisi rakenduvaid päringulimiite on kirjeldatud [sellel veebilehel](https://developer.twitter.com/en/docs/basics/rate-limits). Oma päringulimiitide seisu saate jooksvalt kontrollida meetodi `rate_limit_status()` abil:

In [ ]:
rate_limits = t.application.rate_limit_status()

# Mitu otsipäringut saab veel teha?
rate_limits['resources']['search']['/search/tweets']

In [ ]:
rate_limits = t.application.rate_limit_status()

# Mitu korda saab veel rakenduse-päringuid teha (nt küsida päringulimiite)?
rate_limits['resources']['application']['/application/rate_limit_status']

Muutuja `reset` näitab, kuna toimub järgmine päringulimiitide _algväärtustamine_. Algväärtustamisel viiakse tehtud päringute arvud jälle nulli.

Ajamäärang on antud sekundites ning `datetime` abil saab selle inimloetavaks muuta, nt:

In [ ]:
from datetime import datetime

reset_time = rate_limits['resources']['application']['/application/rate_limit_status']['reset']
human_readable_time = datetime.fromtimestamp( reset_time ).strftime('%Y-%m-%d %H:%M:%S')
human_readable_time

---

## Säutsude kogumine _TweetCat_'i abil

Probleemid, millega puutume kokku eestikeelsete säutsude otsimisel (probleemne keeletuvastus, populaarsete säutsude vähesus, ainult viimase 7 päeva säutsude tagastamine), on esile tulnud ka teiste väikeste keelte jaoks Twitter-i korpuste kogumisel. Ja nende ületamiseks on välja pakutud ka lahendusi. Üheks selliseks lahenduseks on tööriist [_TweetCat_](https://github.com/clarinsi/tweetcat), mis võimaldab säutsude otsimisel / korjamisel rakendada kasutaja-põhist otsingut. 

_TweetCat_-le tuleb anda ette suhteliselt väike arv võtmesõnu (20-40), mis on antud keeles sagedased ning spetsiifilised (st tõenäoliselt pole sagedased mõnes teises keeles). Nende abil leitakse kõigepealt selles keeles kirjutavad Twitter'i kasutajad. Sellele järgneb iteratiivne päringu laiendamine, mille käigus kogutakse juba leitud kasutajate ajajoontelt (ingl _timeline_) säutsud kokku ning seejärel laiendatakse uuritavate kasutajate ringi võttes luubi alla nende sõbrad ja järgijad (eeldades, et nad tõenäoliselt säutsuvad samas keeles).

Kasutajapõhise otsingu eeliseks on, et kasutaja ajajoonelt säutsude korjamisel ei ole "viimase 7 päeva limiiti" -- ühelt kasutajalt saab ka vanemaid säutse. Samuti kannab _TweetCat_ hoolt selle eest, et sõelale jääks võimalikult vähe vales keeles säutse: säutsude korjamisel rakendatakse `langid` keeletuvastust. 

_TweetCat_'i tööpõhimõtteid kirjeldab detailsemalt [see teadusartikkel](https://pdfs.semanticscholar.org/3037/ee591a314481f99902027438eee4c741aa3e.pdf).

### _TweetCat'i_ installimine ja seadistamine

_TweetCat_ on kahjuks selline programm, mis nõuab vana Pythoni versiooni -- Python 2.7-t. Samas ei tähenda see, et peaksid selle kasutamiseks täielikult ümber lülituma vanale Pythonile. Virtuaalkeskkondade võlu ongi selles, et võid luua vana Pythoni versiooniga keskkonna _TweetCat_-i abil andmete korjamiseks, ning andmete töötlemisel saad jälle rakendada harjumuspärast uut Pythoni versiooni. Detailsemalt:

 1. Loo `conda` abil uus virtuaalkeskkond, mis sisaldab Python 2.7.*-t (nt versiooni 2.7.13)   
    ( meeldetuletuseks keskkondade loomise kohta vt 1. praksi juhendit ) 
    
 2. Installi teegid `langid` ( `pip install langid` ) ja `tweepy` ( `pip install tweepy` ) 

 3. Lae oma arvutisse respositooriumist https://github.com/clarinsi/tweetcat _TweetCat_'i projekt 
 
 4. Paki projekt lahti ning ava kaust `1-harvest` 

 5. Ava konfiguratsioonifail ( `hbs.py` ) ja määra:
 
     5.1. Ligipääsuvõtmete väärtused ( `CONSUMER_KEY`, `CONSUMER_SECRET`, `ACCESS_TOKEN`, `ACCESS_TOKEN_SECRET` )   
     5.2. Kasutatav režiim ( `MODE='LANG'` -- keele järgi otsimine )   
     5.3. Huvipakkuvate keelte koodid ( `LANGID_LANG=['et']` -- kui otsida ainult eestikeelseid säutse )  
     5.4. (Vajadusel) võtmesõnade faili nimi muutujas `SEEDS` (kui on oma fail juba olemas)
     
 6. Ava võtmesõnade fail (`'seeds.hbs'` või muutujas `SEEDS` kirjasolev fail) ja lisa sinna võtmesõnad:
 
     * Iga võtmesõna eraldi real  
     * Võtmesõnade arv: 20-..., võiksid olla keskmisest sagedasemad sisusõnad, mis uuritava keele-spetsiifilised

     
### Säutsude kogumine

Kui _TweetCat_ on juba seadistatud, siis säutsude kogumiseks:

 1. Mine Python 2.7.* virtuaalkeskkonnas kataloogi `1-harvest`
 
 2. Käivita säutsude koguja käsurealt (andes talle ette konfiguratsioonifaili nime): 
 
         python harvesting.py hbs
 
 3. Luuakse konfiguratsioonifaili-nimeline kaust ( `hbs` ) ning hakatakse sinna salvestama kogutud säutse kokkupakitud kujul (tekivad `.gz` laiendiga failid);
 
 4. Protsessi katkestamiseks tuleb avada konfiguratsioonifaili-nimeline `.busy` fail ( `hbs.busy` ) ja kirjutada sealne väärtus üle sõnaga `stop`;

### Kogutud säutsude analüüs

Kogutud säutsud on kokkupakitud kujul, `.gz` laiendiga failidena. Mooduli `gzip` abil saate need failid avada ning JSON kujul andmed mällu lugeda, funktsioon `json.loads` aitab aga JSON-i teisendada Pythoni sõnastikuks:

        import gzip
        import json

        fname = '2017-11-13_837713401296076800.gz'

        with gzip.open(fname, 'rb') as f:
            content = f.read()
            # NB! Fail on binaarkujul ja tuleb 
            # dekodeerida
            content = content.decode('utf8')
            tweets = json.loads( content )
            
_Säutsude temaatilisus_. _TweetCat_ kannab küll hoolt selle eest, et säutsud oleksid suure tõenäosusega õiges keeles, aga säutsude temaatilisust ei garanteeri. Seega, kui eesmärgiks on koguda mingil kindlal teemal säutse, peaksid korjatud andmekogu omakorda filtreerima ja kindlal teemal (kindlate märksõnadega) säutsud välja selekteerima.

---

## Säutsude kogumine reaalajas

Kui on tulemas mingi sündmus, millele tõenäoliselt on Twitter'is laialdasem kajastus, siis võib panna püsti ka programmi, mis kuulab Twitter'i säutsuvoogu ning kogub sealt reaalajas huvipakkuvaid säutse. Samuti tasub katsetada säutsude kogumist reaalajas siis, kui eesmärgiks on saada süstemaatiline ülevaade eestikeelsest säutsuruumist mingil ajaperioodil, nt ühe nädala või kuu vältel.

Säutsude reaalajas korjamiseks pakub Twitter välja eraldi veebiteenuse, nn [säutsuvoo filtreerimise API](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/filter-realtime/overview).
Programm, mis on huvitatud säutsude korjamisest reaalajas, võtab esmalt veebiteenusega ühendust ning täpsustab filtreerimise kriteeriumid (nt millistelt kasutajatelt säutse korjatakse, milliseid võtmesõnu säutsud peavad sisaldama, mis keeles peavad säutsud olema jms).
Kui kriteeriumid on paigas, jääb programm ootele ning Twitteri veebiteenus hakkab talle kriteeriumitele vastavaid säutse saatma -- programmi ülesandeks jääb vaid otsustada, mida nendega peale hakata (millised salvestada, millised jätta kõrvale).

Kasutame säutsude reaalajas korjamiseks Pythoni teegi [_tweepy_](https://github.com/tweepy/tweepy) vahendeid. Installimiseks kõigepealt conda käsureale:

    pip install tweepy
    
Kui `tweepy` on olemas, tuleb meil järgmisena luua nn säutsuvoo klass (`Stream` või `StreamingClient`), mis suhtleb Twitteri säutsuvoo veebiteenustega ning otsustab, mida talle saadetud säutsudega teha ning kuidas talitada erijuhtudel (veateated, limiidihoiatused jms). 
Järgnevas näites teeme `Stream` klassi, mis kuulab Twitteri säutsuvoogu (API v1 kaudu), tuvastab eestikeelsed säutsud, prindib välja 5 tuvastatud säutsu, misjärel lõpetab töö.

In [ ]:
from datetime import datetime
import tweepy
import langid

class PrintingStream(tweepy.Stream):

    def __init__(self, *args, **kwargs):
        # Initsialiseerime ülemklassi (Stream)
        super(PrintingStream, self).__init__(*args, **kwargs)
        # Loendame leitud eestikeelseid säutse
        self.et_tweets = 0
    
    def on_status(self, status):
        # Saadeti uus säuts: kontrollime kõigepealt säutsu keelt
        lang = langid.classify(status.text)[0]
        if lang == 'et':
            # Kui paistab olevat eestikeelne,
            # väljastame loomisaja, kasutajanime ja säutsu teksti
            print(status.created_at,'|', status.user.screen_name,'|', status.text)
            self.et_tweets += 1
        # Kui 5 eestikeelset säutsu on väljastatud, lõpetame töö
        if self.et_tweets >= 5:
            print('Lõpetan töö ...')
            self.disconnect()

Säutsuvoog ise ei täpsusta, millistele kriteeriumitele vastavaid säutse tuleb koguda. Meetodi `filter()` abil saame öelda, milliseid säutse me tahame  üles korjata.

Järgnevas näites avame kõigepealt kasutajavõtmete abil uue säutsuvoo (`PrintingStream`) ning rakendame sellel säutsude filtreerimist ja tuvastamist. 
Otsime säutse, mis sisaldavad kõige sagedasemaid eesti keele sõnu (_on, ja, ei, ..._) ning mille keeleks on samuti määratud eesti keel (`'et'`):

In [ ]:
stream = PrintingStream(
    CONSUMER_KEY, CONSUMER_SECRET,
    ACCESS_TOKEN, ACCESS_SECRET
).filter(track=['on', 'ja', 'ei', 'et', 'kui'], languages=["et"])

Eestikeelsete säutsude leidmine muutuks veel täpsemaks tõenäoliselt siis, kui valida hoolikamalt võtmesõnad, mille järgi säutse otsida. 
Ehk siis: tuleks leida sõnad, mis on sagedased küll eesti keeles, aga pigem mittesagedased teistes (Twitter'i) keeltes. 
Seda saaks teha katse-eksituse meetodil: võtta sõnu eesti sõnade sagedusloendist (nt [siit](http://www.cl.ut.ee/ressursid/sagedused1/failid/sonavorm_kahanevas.txt)) ning uurida, kui sageli annab üks või teine sõna eestikeelseid säutse ning kui sageli jääb sõelale valetuvastusi.


_Täiendavad parameetrid_. Lisaks võtmesõnade ja keel(t)e määramisele saab säutsuvoo filtreerimisel veel täpsustada kasutajad, kellelt säutse kogutakse, ning säutsujate geograafilised asukohad (pikkus- ja laiuskraadide järgi). Nende ja teiste päringuparameetrite kohta vt täpsemalt siit: https://developer.twitter.com/en/docs/tweets/filter-realtime/api-reference/post-statuses-filter.html

  * Vt ka inglisekeelset juhendmaterjali _Tweepy_-iga säutsuvoo filtreerimise kohta: https://docs.tweepy.org/en/stable/streaming.html


### Reaalajas säutsudekogumise limiidid (API v1)

Nagu säutsude otsimisel, nii on ka säutsude reaalajas kogumisel omad limiidid. Esiteks, üks Twitter'i kasutaja võib korraga tekitada ainult ühe säutsuvoo filtreerija (`Stream`), mitut filtreerijat samaaegselt kasutada ei ole lubatud.

Teiseks, piiratud on ka filtreerimise kriteeriumite hulgad: korraga tohib filtreerida säutse mitte rohkem kui 400 võtmesõna järgi, 5000 kasutaja järgi ja/või 25 geograafilise asukohapiirangu järgi (v.a tasulise konto korral, mille kohta vt lähemalt [siit](https://developer.twitter.com/en/docs/tweets/filter-realtime/overview)). 

Kolmas oluline piirang puudutab reaalajas filtreerimise tulemusi: veebiteenus tagastab vaid kuni 1% kõigist antud ajahetkel tehtud säutsudest. 
Kui kriteeriumid on liiga avarad ning nendele vastab rohkem kui 1% kõigist säutsudest, siis jäetakse osad säutsud tagastamata (ja nende asemel saadetakse hoiatus, et limiit on ületatud). 
Eestikeelsete säutsude puhul ei tohiks see küll olla probleem -- valdav osa säutsudest tehakse teistes keeltes ning kõik eestikeelsed säutsud peaksid mistahes ajahetkel 1% limiidi alla ära mahtuma (vähemalt kõik saksakeelsed säutsud on mahtunud 1% limiidi alla, väidab [see teadustöö](http://www.lrec-conf.org/proceedings/lrec2014/pdf/1146_Paper.pdf) -- suure tõenäosusega mahuvad siis ka eestikeelsed). 

### _Säutsude reaalajas filtreerimine -- kas kasutada või mitte?_

Võrreldes mineviku-säutsude otsimisega või TweetCat'i abil kogumisega pakub säutsude reaalajas kogumine kõige süstemaatilisemat viisi säutsude korjamiseks: kui filtreerimiskriteeriumid on hästi valitud, siis võib kätte saada *praktiliselt kõik mingil ajaperioodil tehtud eestikeelsed säutsud*. 

Mündi teine külg on aga see, et nii saab koguda vaid tuleviku-säutse ning reaalajas kogumine on ka üksjagu keerulisem kui mineviku-säutsude otsimine või TweetCat'iga kogumine. 

 * Kui filtreerimise näites kogusime säutse reaalajas paarikümne sekundi jooksul, siis sellise programmi jooksutamine nädalate või isegi kuude kaupa nõuab kõigepealt stabiilse netiühendusega ja pikaajalist töötamist võimaldavat keskkonda. Tõenäoliselt oleks seda kõige parem teha mitte isiklikus sülearvutis, vaid kusagil serveris -- laadida programm üles ja "panna tiksuma";
 * Kui eesmärgiks on saada kätte kõik eestikeelsed säutsud, siis heade filtreerimiskriteeriumite valimine nõuab tõenäoliselt üksjagu katsetamist -- milliseid võtmesõnu valida, kuidas kombineerida võtmesõnade järgi filtreerimist kasutajate ja asukohtade järgi filtreerimisega, kuidas veenduda / testida, et 95% ja rohkem eestikeelsetest säutsudest leitakse üles jne. Natuke saab siin eeskuju võtta [sellest uurimustööst](http://www.lrec-conf.org/proceedings/lrec2014/pdf/1146_Paper.pdf), kus lahendati sama ülesannet saksakeelsete säutsude puhul;
 * Kuna programm pannakse käima pikaks ajaks, on tarvis ka läbi mõelda (ja implementeerida) piirisituatsioonid (_a la_ mida teha erinevate hoiatuste korral, mida server võib saata, ning kuidas (automaatselt) taastada katkenud ühendus);

Kokkuvõttes: säutsude reaalajas kogumise liides on võimas tööriist, aga selle kasutamaõppimine ja kasutamine nõuavad üksjagu  tööd ja aega.